In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

%cd "/content/gdrive/My Drive/Colab Notebooks/Age detection/"

Mounted at /content/gdrive/
/content/gdrive/My Drive/Colab Notebooks/Age detection


In [ ]:
import pandas as pd
import numpy as np
from fastai.vision import *
import os
import pickle
from sklearn import preprocessing 
import torch
import torch.nn as nn
from torch.optim import lr_scheduler

## Data augmentation

In [ ]:
!unzip "./Age-Detection-Actors-Challenge/data/train_DETg9GD.zip" -d "./Age-Detection-Actors-Challenge/data/Origen_data" #Unzip the data

In [ ]:
Labels = pd.read_csv("./Age-Detection-Actors-Challenge/data/train.csv")

In [ ]:
#Function to apply transformations

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

tfms = get_transforms(max_rotate=25)

#Function to generate new images

def data_aug_one(ex_img, name,qnt, path_hr):
   
   for lop in range(0,qnt):
      image_name = str(lop) + '_' + name
      dest = path_hr+"/"+image_name
      new_img = open_image(ex_img)
      new_img_fin = new_img.apply_tfms(tfms[0], new_img, xtra={tfms[1][0].tfm: {"size": 224}}, size=224)
      new_img_fin.save(dest)
     


In [ ]:
OLDS = Labels[Labels["Class"]=="OLD"] #List of ID of class OLD
List_old = OLDS["ID"].to_list() #Final List of names of images
New_list = ["./Age-Detection-Actors-Challenge/data/Origen_data/Train/" + item for item in List_old] #Add complete plat
print(len(New_list))

2396


In [ ]:
qnt = 3 #Number of images to generate
path_hr = "./Age-Detection-Actors-Challenge/data/Origen_data/New_data_old_new" #Final Path
epoch = 0 #sub id file generated

for i in New_list:
  name = os.path.basename(i)
  data_aug_one(i,name,qnt,path_hr)
  if epoch % 100 == 0 :
    print("Vamos en la imagen : {}".format(epoch))
  epoch+=qnt
name = os.path.basename(New_list[2395])
data_aug_one(New_list[2395],name,qnt,path_hr)

In [ ]:
#Same proccess for class YOUNG
YOUNG = Labels[Labels["Class"]=="YOUNG"] 
List_young = YOUNG["ID"].to_list()
List_young = List_young[1500:2000]

In [ ]:
New_list_young = ["./Age-Detection-Actors-Challenge/data/Origen_data/Train/" + item for item in List_young]
len(New_list_young)

500

In [ ]:
qnt = 2 #Two imagen in this clase
path_hr = "./Age-Detection-Actors-Challenge/data/Origen_data/New_data_Young"

for i in New_list_young:
  name = os.path.basename(i)
  data_aug_one(i,name,qnt,path_hr)

In [ ]:
Labels_new = Labels.copy()

In [ ]:
Listfiles_old = os.listdir("./Age-Detection-Actors-Challenge/data/Origen_data/New_data_Old") #Files of old category
Listfiles_young = os.listdir("./Age-Detection-Actors-Challenge/data/Origen_data/New_data_Young") #Files of Young category

In [ ]:
#Add ID of new imagen to final list of train
for old in Listfiles_old:
  df_length = len(Labels_new)
  Labels_new.loc[df_length] = [old , "OLD"]

In [ ]:
for young in Listfiles_young:
  df_length = len(Labels_new)
  Labels_new.loc[df_length] = [young , "YOUNG"]

In [ ]:
Labels.groupby(['Class']).count() #Before Data augmentation

,ID
Class,
MIDDLE,10804
OLD,2396
YOUNG,6706


In [ ]:
Labels_new.groupby(['Class']).count() #After data augmentation

,ID
Class,
MIDDLE,10804
OLD,10480
YOUNG,10706


In [ ]:
Labels_new.shape , Labels.shape

((31990, 2), (19906, 2))

In [ ]:
#Move the files to one folder

import shutil

source_dir = "./Age-Detection-Actors-Challenge/data/Origen_data/New_data_Young"
target_dir = './Age-Detection-Actors-Challenge/data/Origen_data/Train'
    
file_names = os.listdir(source_dir)

for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

In [ ]:
source_dir = "./Age-Detection-Actors-Challenge/data/Origen_data/New_data_Young"
target_dir = './Age-Detection-Actors-Challenge/data/Origen_data/Train'
    
file_names = os.listdir(target_dir)
len(file_names)

31990